In [1]:
import numpy as np
import pandas as pd
import sys

In [2]:
data=pd.read_csv("../../datasets/iris.csv")
data.head()

,5.1,3.5,1.4,0.2,Iris-setosa
0,4.9,3.0,1.4,0.2,Iris-setosa
1,4.7,3.2,1.3,0.2,Iris-setosa
2,4.6,3.1,1.5,0.2,Iris-setosa
3,5.0,3.6,1.4,0.2,Iris-setosa
4,5.4,3.9,1.7,0.4,Iris-setosa


In [3]:
data['Iris-setosa']=data['Iris-setosa'].astype('category')
data['Iris-setosa']=data['Iris-setosa'].cat.codes
y=data['Iris-setosa'].as_matrix()
data=data.drop('Iris-setosa',axis=1)
x=data.as_matrix()

In [4]:
squarer = lambda x:(float)("{:.3f}".format(x))
vfunc = np.vectorize(squarer)
x=vfunc(x)



In [28]:
print(np.min(x[:,2]))

1.0


In [8]:
n_classes=np.unique(y, return_counts=False).shape[0]

In [32]:
class DecisionTree:
    class Node:
        def __init__(self,k,alpha,impurity,label,id1):
            self.left={}
            self.right={}
            self.parent={}
            self.impurity={}
            self.alpha=alpha
            self.k=k
            self.label=label
            self.id=id1
            
            
            
    def __init__(self,imp,depth,n_classes):
        self.root={}
        self.imp=imp
        self.depth=depth
        self.n_classes=n_classes
      
        
    
    def I(self,x,y):
        #print("I")
        if self.imp=='gini':
            return self.gini(x,y)
        elif self.imp=='misscl':
            return self.misscl(x,y)
        elif self.imp=='entropy':
            return self.entropy(x,y)
        elif self.imp=='regress':
            return self.regressionCost(x,y)
        
        
    def fit(self,x,y):
        xd=[]
        for i in range(x.shape[1]):
            #print(x[:,i])
            tmp=list(set(x[:,i]))
            tmp.sort()
            xd.append(list(map(lambda x:(float)("{:.3f}".format(x)),tmp)))
            
        
        Ix=self.I(x,y)
        self.node_split(x,y,None,True,0,xd,Ix,"0")
        
    def calculateP(self,x,y):
        pi=[]
        
        for i in range(self.n_classes):
            if(len(y)==0):
                pi.append(0)
            else:
                pi.append(len(y[y==i])/len(y))
        return pi
        
    def misscl(self,x,y):
        pi=np.array(self.calculateP(x,y))
        return 1-np.argamax(pi)
    
    def entropy(self,x,y):
        
        pi=np.array(self.calculateP(x,y))
        #print("pi=",pi)
        sm=0
        for i in range(pi.shape[0]):
            if(pi[i]!=0):
                sm+=pi[i]*np.log(pi[i])
        #print("sum=",sm)
        return -sm
     
    
    def gini(self,x,y):
        pi=np.array(self.calculateP(x,y))
        return np.sum(pi*(1-pi))
        
    def regress(self,x,y):
        return np.var(y)
        
    
    def node_split(self,x,y,nd,b,depth,xd,Ix,st):
    
        
        print("St=",st," depth=",depth," b=",b," Ix=",Ix)
        if depth==self.depth or Ix==0 :
            
            label={}
            if self.imp=='regers':
                label=y.mean()
            else:
                unique, counts = np.unique(y, return_counts=True)
                p=dict(zip(unique, counts))
              
                label=max(p, key=p.get)
                print("label=",label)
            node=self.Node(None,None,Ix,label,st)
            return
       
        mn=10000000.0
        #print("mn=",mn)
        kk=0
        ii=0
        xty={}
        xtn={}
        yty={}
        ytn={}
        xy={}
        xn={}
        for k in range(x.shape[1]):
            
            #print("k=",k," len=",len(xd[k]))
            
            for i in range(len(xd[k])):
                #print("i=",i)
                xty=x[(x[:,k]>xd[k][0]) & (x[:,k]<=xd[k][i])]
                xtn=x[(x[:,k]>=xd[k][i]) & (x[:,k]<xd[k][len(xd[k])-1])]
                yty=y[(x[:,k]>xd[k][0] )& (x[:,k]<=xd[k][i])]
                ytn=y[(x[:,k]>=xd[k][i]) & (x[:,k]<xd[k][len(xd[k])-1])]
                
                p=len(xty)/len(x)
                #print(p)
                
                Ixty=self.I(xty,yty)
                Ixtn=self.I(xtn,ytn)
                #print(" Ixty=",Ixty," Ixtn=",Ixtn," p=",p," ",str(p*Ixty+(1-p)*Ixtn))
                
                if(mn>p*Ixty+(1-p)*Ixtn and Ix-(p*Ixty+(1-p)*Ixtn >0)):
                    gy=Ixty
                    kk=k
                    mn=p*Ixty+(1-p)*Ixtn
                    ii=i
                    gn=Ixtn
                    xy=np.copy(xty)
                    xn=np.copy(xtn)
                    yy=np.copy(yty)
                    yn=np.copy(ytn)
                
        #print("I(x)=",Ix," I(xty)=",gy," I(xny)=",gn," p=",p," kk=",kk," ii=",ii," mn=",mn)
        #print(len(xd[kk]))
        #print("")
        #print("")
        node=self.Node(kk,xd[kk][ii],Ix,None,st)
        if nd==None:
            self.root=node
        
        if b and nd!= None:
            nd.left=node
        elif nd!= None:
            nd.right=node
        node.parent=nd
        
    
        xd1=np.copy(xd)
       # print("shape=",xy.shape[0],xn.shape[0])
        print(node.k,node.alpha)
        if  xy.shape[0]>0:
            xd1[kk]=xd[kk][:ii]
            self.node_split(xy,yy,node,True,depth+1,xd1,gy,st+"t")
        if  xn.shape[0]>0:
            xd1[kk]=xd[kk][ii+1:]
            self.node_split(xn,yn,node,False,depth+1,xd1,gn,st+"f")
        
    def predict(self,x):
        node=self.root
        y=[]
        for i in range(x.shape[0]):
            while(node!=None):
                print(node)
                tmp=node.label
                
                if x[i,node.k]<=node.alpha:
                    node=node.left
                elif x[i,node.k]>node.alpha:
                    node=node.right
                
            y.append(tmp)
        y=np.array(y)
        return y
                    
                    
                
        
        

In [33]:
decisiontree=DecisionTree('entropy',50,n_classes)

In [34]:
decisiontree.fit(x,y)

St= 0  depth= 0  b= True  Ix= 1.09856714386
2 3.0
St= 0t  depth= 1  b= True  Ix= 0.0996233894662
2 1.0
St= 0tf  depth= 2  b= False  Ix= -0.0
label= 0
St= 0f  depth= 1  b= False  Ix= 0.69309616449
3 1.7
St= 0ft  depth= 2  b= True  Ix= 0.308495450831
3 1.3
St= 0ftt  depth= 3  b= True  Ix= -0.0
label= 1
St= 0ftf  depth= 3  b= False  Ix= 0.304636097349
2 4.9
St= 0ftft  depth= 4  b= True  Ix= -0.0
label= 1
St= 0ftff  depth= 4  b= False  Ix= 0.673011667009
2 5.0
St= 0ftfff  depth= 5  b= False  Ix= -0.0
label= 2
St= 0ff  depth= 2  b= False  Ix= 0.184907399168
3 1.9
St= 0fft  depth= 3  b= True  Ix= -0.0
label= 2
St= 0fff  depth= 3  b= False  Ix= -0.0
label= 2


In [412]:
predict=decisiontree.predict(x)

{}


AttributeError: 'dict' object has no attribute 'label'

In [189]:
mn=-0
b=0
b==mn

True

In [206]:
import numpy as np
x = np.array([[1,2],[3,4]])
squarer = lambda t: t ** 2
vfunc = np.vectorize(squarer)
vfunc(x)

array([[ 1,  4],
       [ 9, 16]])

In [260]:
o=np.random.rand(4,2)

o[2,:]=[1,2]

In [272]:
p=list([list([1,2,3,4]),list([4,5,6,7,8])])
print(p)
def func(p):
    p1=np.copy(p)
    
    p1[1]=p1[1][:2]
    print(p1)
func(p)
#print(tmp)
#p[1]=tmp
print(p)

[[1, 2, 3, 4], [4, 5, 6, 7, 8]]
[[1, 2, 3, 4] [4, 5]]
[[1, 2, 3, 4], [4, 5, 6, 7, 8]]


In [290]:
x=np.random.randint(100,size=(100,4))

In [296]:
a=
b=

In [304]:
(x[:,2]<40) and (x[:,2]<20)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [391]:
def t():
    

2